In [21]:
import cv2
import os

In [ ]:
def extract_video_frames(video_path, skip_seconds=(2, 2), frame_interval=10, output_name="", output_dir="train_samples"):
    """
    Extract frames from a video file, skipping the first and last few seconds,
    and saving them as images in a specified directory.
    :param video_path: Path to the input video file.
    :param skip_seconds: Tuple of seconds to skip at the start and end of the video.
    :param frame_interval: Interval at which frames are saved (0 means save all frames).
    :param output_name: Name for the output directory and image files.
    """
    
    if not video_path or not isinstance(video_path, str):
        raise ValueError("video_path must be a non-empty string.")
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video file not found: {video_path}")
    if not isinstance(skip_seconds, tuple) or len(skip_seconds) != 2:
        raise ValueError("skip_seconds must be a tuple of two values (start_skip, end_skip).")
    if not isinstance(frame_interval, int) or frame_interval < 0:
        raise ValueError("frame_interval must be a non-negative integer.")
    if not isinstance(output_name, str) or not output_name:
        raise ValueError("output_name must be a non-empty string.")
    
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Calculate start and end frames after skipping initial/final seconds
    start_frame = int(skip_seconds[0] * fps)
    end_frame = total_frames - int(skip_seconds[1] * fps)
    if start_frame < 0 or end_frame < 0:
        raise ValueError("Skip seconds result in negative frame indices.")
    elif start_frame >= total_frames or end_frame > total_frames:
        raise ValueError("Skip seconds exceed total video duration.")
    elif start_frame >= end_frame:
        raise ValueError("Start frame must be less than end frame after skipping seconds.")

    # Create output directory
    full_output_dir = f"Data/{output_dir}/{output_name}/original"
    os.makedirs(full_output_dir, exist_ok=True)
    
    # Find the last image number in the directory
    existing_files = [f for f in os.listdir(full_output_dir) if f.startswith(output_name) and f.endswith('.png')]
    if existing_files:
        last_number = max([int(f.replace(output_name, "").replace(".png", "")) for f in existing_files])
        saved_count = last_number + 1
    else:
        saved_count = 0
    
    current_frame = 0

    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        if start_frame <= current_frame < end_frame:
            if (frame_interval == 0) or (current_frame - start_frame) % frame_interval == 0:
                filename = os.path.join(full_output_dir, f"{output_name}{saved_count}.png")
                cv2.imwrite(filename, frame)
                saved_count += 1
        current_frame += 1

    cap.release()

    # Print analysis
    print("=== VIDEO ANALYSIS ===")
    print(f"Total frames: {total_frames}")
    print(f"Total frames (after skipped seconds): {end_frame - start_frame}")
    print(f"Images saved in this run: {saved_count - (last_number + 1 if "last_number" in locals() else 0)}")
    print(f"Total images in directory: {saved_count}")
    print(f"Frame size (width x height): {frame_width} x {frame_height}")


In [25]:
extract_video_frames("video.mp4", skip_seconds=(2, 2), output_name="img")

=== VIDEO ANALYSIS ===
Total frames: 305
Total frames (after skipped seconds): 187
Images saved in this run: 19
Total images in directory: 57
Frame size (width x height): 478 x 850
